## Welcome to MR simulation 
In this interactive notebook I'll guide you through using 2 powerfull github repositories and code to simulate MR signal acquisition. </br>
Credit to Eva Alonso Ortiz and Charles Pageot for *susceptibility-to-fieldmap-fft* 

In [1]:
import numpy as np
import nibabel as nib
import os
import matplotlib.pyplot as plt

## Importing, cloning and installing

In [2]:
!git clone https://github.com/shimming-toolbox/tissue-to-MRproperty.git

fatal: destination path 'tissue-to-MRproperty' already exists and is not an empty directory.


In [2]:
%cd tissue-to-MRproperty
!pip install .
%cd ..

C:\Users\User\msc_project\Image-processing-strategies\MRsim\tissue-to-MRproperty
Processing c:\users\user\msc_project\image-processing-strategies\mrsim\tissue-to-mrproperty
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for tissue-to-MRproperty: filename=tissue_to_MRproperty-0.0.0-py3-none-any.whl size=29855 sha256=da875df9b4892841e52c736b72238e49db61e47bd86463523cd86ade6475c6a8
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\ce\8f\65\c868b29f9cf7cbb6272746dda3ed0e1945ec499eb343897991
Successfully built tissue-to-MRproperty
  Attempting uninstall: tissue-to-MRproperty
    Found existing installation: tissue-to-MRproperty 0.0.0
    Uninstalling tissue-to-MRproperty-0.0.0:
      Successfully uninstalled tissue-to-MRproperty-0.0.0
C:\Users\User\msc_project\Image-processing-strategies\MRsim


In [7]:
!git clone https://github.com/shimming-toolbox/susceptibility-to-fieldmap-fft.git

Cloning into 'susceptibility-to-fieldmap-fft'...


In [3]:
%cd susceptibility-to-fieldmap-fft
!pip install .
%cd ..

C:\Users\User\msc_project\Image-processing-strategies\MRsim\susceptibility-to-fieldmap-fft
Processing c:\users\user\msc_project\image-processing-strategies\mrsim\susceptibility-to-fieldmap-fft
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for susceptibility-to-fieldmap-fft: filename=susceptibility_to_fieldmap_fft-0.0.0-py3-none-any.whl size=15984 sha256=2ef42b7f6b8aae6e5613805acfefa98002f4eae67cb3567213666c69b30c94b4
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\89\fe\26\78bb69b2c55cd65a6250f1b426eed9e7c2459733c05463bafe
Successfully built susceptibility-to-fieldmap-fft
  Attempting uninstall: susceptibility-to-fieldmap-fft
    Found existing installation: susceptibility-to-fieldmap-fft 0.0.0
    Uninstalling susceptibility-to-fieldmap-fft-0.0.0:
      Successfully uninstalled susceptibility-to-fieldmap-fft-0.0.0
C:\Users\User\msc_project\Image-processing-strategies\MRsim


## Prepare for Simulation
For the simulation of MR signals we can guide ourselves with a simplified equation for simulating GRE acquisition. </br>
$$ \text{protonDensity} \cdot \sin(\text{FA}) \cdot \exp\left(-\frac{TE}{T2^*} - \text{sign} \cdot i \cdot \gamma \cdot \delta B_0 \cdot TE\right) $$

This equation requires as input the *flip angle* and the list of *echo times*, and it requires for simulation purposes:
- T2star volume
- PD volume
- delta B0
The delta B0 requires the susceptibility distribution volume. </br>

For the 3 volume requirements we use tissue to MR property repository whereas for deltaB0 we use the susceptibility to fieldmap repository.

Tissue to MR repo will automatically create 3 folders: data, output and simulation. This is for them to store any heavy information as Nifti images of processing that will be ignored by git. </br>
When you run the converter you'll find everything under *output* </br>
I recommend that you copy and paste this in your commands to your terminal because a new feature was implemented called *check pixel integrity* that checks the pixels have correct label ID before doing the conversion. This function will talk with you to correct the values telling you where the mistake pixel is located so you can open a viewer like ITK or FSLeyes and check what value it should have. This mistake arrises while adding labels and is fairly common, hence the addition of the method.

In [ ]:
!tissue_to_mr ratatouille.nii.gz -t sus -s TotalSeg_CT -v mod2 sus_volume.nii.gz

In [15]:
!tissue_to_mr ratatouille.nii.gz -t t2 -s TotalSeg_CT -v mod2 t2s_volume.nii.g

^C


In [ ]:
!tissue_to_mr ratatouille.nii.gz -t pd -s TotalSeg_CT -v mod2 pd_volume.nii.gz

One very important argument we need to calculate is the deltaB0, for which we use Susceptibility to Fieldmap fft repository. </br>
Which estimates the magnetic field perturbation that arises when an object is placed withing a magnetic field. </br>
After pip installing the repo run the command compute_fieldmap and give it the susceptibility distribution volume created before

In [34]:
!compute_fieldmap -i "data/sus_volume.nii.gz" -o "data/fieldmap.nii.gz"


Start
Susceptibility distribution loaded
Fieldmap simulated
Saving to NIfTI format
End. Runtime: 67.59 seconds


## Simulation
Now we have all the necessary Nifti files to simulate MR signal acquisition. The folder *utils* contains the necessary code for simulating with comments.

In [2]:
%cd utils
from sim_functions import optimize_measurement
%cd ..

C:\Users\User\msc_project\Image-processing-strategies\MRsim\utils
C:\Users\User\msc_project\Image-processing-strategies\MRsim


In [3]:
# Edit the echo time (or list of echo times) and the flip angle:
TE_list = [0.008, 0.0095]#, 0.011, 0.0125, 0.014, 0.0155]
flip_ang = 24
# The last requirement for the code is the dimensions, which can be easily acquired from the volume you work with
# The dimensions needed because we will create a 4D array, where we will store the echo time data and we want to maintain the initial 3D dimensions.
data = nib.load("data/ratatouille_crop.nii.gz")
np_data = data.get_fdata()
dimensions = np.array(np_data.shape)
dimensions

array([109, 129, 218])

In [4]:
# Loading the data
pd= nib.load("data/pd_volume_crop.nii.gz")
pd_vol = pd.get_fdata()
t2s = nib.load("data/t2s_volume_crop.nii.gz")
t2s_vol = t2s.get_fdata()
deltaB0 = nib.load("data/fieldmap_crop.nii.gz")
dB0 = deltaB0.get_fdata()
magnitude, phase = optimize_measurement(pd_vol, t2s_vol,dimensions, dB0,flip_ang,TE_list) # By default, B0 is 3 Tesla

Starting optimize_measurement
Processing TE[0] = 0.008
Starting optimized_signal
Finished optimized_signal
mag shape: (109, 129, 218), phase_arr shape: (109, 129, 218)
Processing TE[1] = 0.0095
Starting optimized_signal
Finished optimized_signal
mag shape: (109, 129, 218), phase_arr shape: (109, 129, 218)
Finished optimize_measurement


In [5]:
# Finally we save the magnitude and the phase to analyze them
temp_mag = nib.Nifti1Image(magnitude, affine=data.affine)
temp_ph = nib.Nifti1Image(phase, affine=data.affine)
path_mag = os.path.join('output','sim_mag.nii.gz')
path_ph = os.path.join('output','sim_phase.nii.gz')
nib.save(temp_mag, path_mag)
nib.save(temp_ph, path_ph)